In [1]:
# Импорты
import pandas as pd
import joblib
import numpy as np
import random
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score, hamming_loss, roc_auc_score
from typing import Dict, List, Union
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_selection import VarianceThreshold
from nltk.util import ngrams
random.seed(42)
np.random.seed(42)
pd.options.mode.chained_assignment = None

In [2]:
df = pd.read_parquet("df_l_fin.parquet")

In [3]:
df.head()

,author,publication_date,hubs,comments,views,url,reading_time,individ/company,bookmarks_cnt,text_length,tags_tokens,title_tokens,rating_new,text_tokens,text_pos_tags
0,complex,2009-08-03 14:34:35+00:00,GTD,67,6800,https://habr.com/ru/articles/66091/,2.0,individual,25.0,2027,['лень' 'учись' 'работать' 'самомотивация' 'мо...,['лечение' 'приступ' 'лень'],4.0,['лишать' 'девственность' 'бложик' 'происходит...,"[NOUN, VERB, NOUN, DET, NOUN, ADV, SCONJ, PRON..."
1,popotam2,2009-07-15 20:24:31+00:00,GTD,13,3100,https://habr.com/ru/articles/64586/,1.0,individual,6.0,424,['развитие' 'работоспособность' 'организация' ...,['организация' 'рабочий' 'время' 'помощь' 'цвет'],1.0,['предлагать' 'вариант' 'сделать' 'организован...,"[VERB, ADV, NUM, NOUN, VERB, PRON, ADV, ADJ, C..."
2,COOL_ALMANAH,2009-05-13 11:37:50+00:00,GTD,22,1300,https://habr.com/ru/articles/59477/,1.0,individual,6.0,1084,['flash' 'работа' 'время' 'контроль' 'шахматны...,['контроль' 'потерять' 'время' 'flash' 'обновл...,4.0,['таймер' 'обновиться' 'надеяться' 'удобный' '...,"[NOUN, VERB, SPACE, ADV, PRON, VERB, AUX, ADJ,..."
3,ZakharS,2009-05-14 11:13:12+00:00,GTD,314,6700,https://habr.com/ru/articles/59584/,5.0,individual,104.0,7852,['карьера' 'it' 'финансы' 'рост'],['достигнуть' 'потолок' 'зарплата'],147.0,['предупредить' 'несмотря' 'экран' 'текст' 'ст...,"[VERB, ADV, VERB, PRON, ADV, ADP, NUM, NOUN, N..."
4,Kerf,2009-03-19 14:44:04+00:00,GTD,3,892,https://habr.com/ru/articles/54970/,6.0,individual,16.0,8872,['бизнес' 'корпорация' 'вебдизайн' 'контент'],['вещь' 'знать' 'клиент'],3.0,['когдалибо' 'чувствовать' 'бесконечно' 'повто...,"[PRON, PRON, VERB, SCONJ, PRON, ADV, VERB, DET..."


In [4]:
df.hubs.value_counts()

hubs
Чулан                                                29244
Я пиарюсь                                             6217
IT-компании                                           5805
Информационная безопасность                           4650
Веб-разработка                                        3014
                                                     ...  
Математика, Транспорт                                    1
Блог компании RUVDS.com, Open source, Карьера ...        1
Блог компании Karuna, Управление разработкой, ...        1
Open source, Python, API, Софт, Искусственный ...        1
C++, Проектирование и рефакторинг, Управление ...        1
Name: count, Length: 77273, dtype: Int64

# Предобработка

## Модификации

In [5]:
# Уберем слишком короткие (неинформативные) статьи
display(df['text_length'].describe())
df = df[df['text_length']>100].copy()
df['text_length'].describe()

count    284895.000000
mean       7571.855771
std        7980.117174
min           1.000000
25%        2104.000000
50%        5514.000000
75%       10315.000000
max      197359.000000
Name: text_length, dtype: float64

count    282640.000000
mean       7631.863576
std        7983.445597
min         101.000000
25%        2168.000000
50%        5575.000000
75%       10372.000000
max      197359.000000
Name: text_length, dtype: float64

In [6]:
# Выделение шкалы оценок на основе рейтинга статьи,
# Это будет таргет для предсказания оценки статьи
neg_d = df[df['rating_new']<0]['rating_new'].describe()
pos_d = df[df['rating_new']>0]['rating_new'].describe()

def rating_func(row):
    rate = row['rating_new']

    if pos_d['25%'] >= rate >= neg_d['75%']:
        return('neutral')
    
    elif pos_d['75%'] >= rate > pos_d['25%']:
        return('positive')
    
    elif rate > pos_d['75%']:
        return('very positive')
    
    elif neg_d['75%'] > rate >= neg_d['25%']:
        return('negative')
    
    elif neg_d['25%'] > rate:
        return('very negative')
    
df['rating_level'] = df.apply(rating_func, axis=1)

In [7]:
# сохранение дф для создания N-грам
df_wth_tokens = df.copy()

# Сразу удалим ненужные столбцы для облегчения вычислений
df_wth_tokens.drop(columns=['tags_tokens', 'title_tokens'], inplace=True)

In [8]:
# Извлекаем уникальные метки из тегов
unique_labels = set()
df_wth_tokens['hubs'].str.split(', ').apply(unique_labels.update)  # Собираем уникальные метки
label_to_index = {label: idx for idx, label in enumerate(sorted(unique_labels))}  # Маппинг меток в индексы

# Преобразуем строки в списки индексов
df_wth_tokens['hubs_encoded'] = df_wth_tokens['hubs'].apply(lambda x: [label_to_index[label] for label in x.split(', ')])

In [9]:
df_wth_tokens.head(1)

,author,publication_date,hubs,comments,views,url,reading_time,individ/company,bookmarks_cnt,text_length,rating_new,text_tokens,text_pos_tags,rating_level,hubs_encoded
0,complex,2009-08-03 14:34:35+00:00,GTD,67,6800,https://habr.com/ru/articles/66091/,2.0,individual,25.0,2027,4.0,['лишать' 'девственность' 'бложик' 'происходит...,"[NOUN, VERB, NOUN, DET, NOUN, ADV, SCONJ, PRON...",neutral,[80]


In [10]:
# Функция для предсказания метрик precision, recall, f1-score и hamming-loss
def calculate_metrics(
    y_test: Union[List[int], np.ndarray],
    y_pred: Union[List[int], np.ndarray],
    average: str = '',
    zero_division: int = 0) -> Dict[str, float]:
    """
    Вычисление метрик precision, recall, f1-score и hamming-loss

    Параметры:
        y_test: Истинные значения
        y_pred: Предсказанные значения
        average: Метод усреднения для метрик precision, recall и f1-score
                       Может быть 'micro', 'macro'или 'weighted'
        zero_division: Обработка деления на ноль в метриках (0 или 1).

    Возвращает:
        Dict: Словарь с метриками
    """
    precision = round(precision_score(y_test, y_pred, average=average, zero_division=zero_division), 4)
    recall = round(recall_score(y_test, y_pred, average=average, zero_division=zero_division), 4)
    f1 = round(f1_score(y_test, y_pred, average=average, zero_division=zero_division), 4)
    hamming = round(hamming_loss(y_test, y_pred), 4)

    metrics = {'Precision': precision, 'Recall': recall,
               'F1-Score': f1, 'Hamming Loss': hamming}
    return metrics

## Сэмплирование

In [11]:
# Сэмпл данных - 10% от всего объёма
sample_df = df_wth_tokens.sample(frac=0.1, random_state=42)

# Преобразование hubs_ecoded в формат матрицы с уникальными метками хабов
mlb = MultiLabelBinarizer()
y_multi = mlb.fit_transform(sample_df['hubs_encoded'])

# Удаляем метки, которые встречаются в менее чем 1% случаев
selector = VarianceThreshold(threshold=0.01)
y_multi_reduced = selector.fit_transform(y_multi)

In [12]:
y_multi_reduced.shape

(28264, 56)

# Классификация

## обработка html тегов

кривая строка -> ['слово', 'слово2']

In [13]:
df_wth_tokens.head(1)['text_tokens'].values

<StringArray>
["['лишать' 'девственность' 'бложик' 'происходить' 'сегодня' 'понедельник'\n 'день' 'неделя' 'крокодил' 'ловится' 'расти' 'кокос' 'закину'\n 'соответствующий' 'пост' 'самомотивация' 'период' 'жизнь' 'инициативный'\n 'неугомонный' 'трудоголик' 'смотреть' 'подумаешь' 'периодически'\n 'происходить' 'нападать' 'тотальный' 'апатия' 'лень' 'происходить'\n 'неимение' 'возможность' 'устроить' 'выходной' 'неделя' 'подряд'\n 'происходить' 'избавиться' 'пост' 'написать' 'учись' 'работать'\n 'мотивация' 'заставлять' 'работать' 'избавляться' 'состояние' 'работать'\n 'хотеться' 'человек' 'инициативный' 'неугомонный' 'трудоголик'\n 'гордиться' 'временами' 'день' 'ночь' 'сутками' 'работа' 'работать'\n 'попадать' 'ступор' 'мозг' 'напрочь' 'отказываться' 'чтолибо' 'думать'\n 'соответственно' 'чтото' 'делать' 'получаться' 'плохо' 'голова'\n 'вериться' 'фраза' 'работать' 'делать' 'хотеться' 'озвучить' 'многих'\n 'топика' 'хабре' 'лекарство' 'помогать' 'отключить' 'интернет'\n 'представить' '

In [14]:
import re

def extract_words(input_string):
    # Заменяем символы '[', ']', '"' на пробелы
    
    # Извлекаем слова, используя регулярное выражение
    words = re.findall(r"(\b\w+\b)", input_string)
    return words

# Пример использования
input_string = 'Список: ["apple", "banana", [мир]]'
words = extract_words(input_string)
print(words)

['Список', 'apple', 'banana', 'мир']


In [15]:
df_wth_tokens['clear_text'] = df_wth_tokens['text_tokens'].apply(extract_words)

In [16]:
df_wth_tokens.head(2)

,author,publication_date,hubs,comments,views,url,reading_time,individ/company,bookmarks_cnt,text_length,rating_new,text_tokens,text_pos_tags,rating_level,hubs_encoded,clear_text
0,complex,2009-08-03 14:34:35+00:00,GTD,67,6800,https://habr.com/ru/articles/66091/,2.0,individual,25.0,2027,4.0,['лишать' 'девственность' 'бложик' 'происходит...,"[NOUN, VERB, NOUN, DET, NOUN, ADV, SCONJ, PRON...",neutral,[80],"[лишать, девственность, бложик, происходить, с..."
1,popotam2,2009-07-15 20:24:31+00:00,GTD,13,3100,https://habr.com/ru/articles/64586/,1.0,individual,6.0,424,1.0,['предлагать' 'вариант' 'сделать' 'организован...,"[VERB, ADV, NUM, NOUN, VERB, PRON, ADV, ADJ, C...",neutral,[80],"[предлагать, вариант, сделать, организованный,..."


In [17]:
df_wth_tokens.head(1)['clear_text'].values

array([list(['лишать', 'девственность', 'бложик', 'происходить', 'сегодня', 'понедельник', 'день', 'неделя', 'крокодил', 'ловится', 'расти', 'кокос', 'закину', 'соответствующий', 'пост', 'самомотивация', 'период', 'жизнь', 'инициативный', 'неугомонный', 'трудоголик', 'смотреть', 'подумаешь', 'периодически', 'происходить', 'нападать', 'тотальный', 'апатия', 'лень', 'происходить', 'неимение', 'возможность', 'устроить', 'выходной', 'неделя', 'подряд', 'происходить', 'избавиться', 'пост', 'написать', 'учись', 'работать', 'мотивация', 'заставлять', 'работать', 'избавляться', 'состояние', 'работать', 'хотеться', 'человек', 'инициативный', 'неугомонный', 'трудоголик', 'гордиться', 'временами', 'день', 'ночь', 'сутками', 'работа', 'работать', 'попадать', 'ступор', 'мозг', 'напрочь', 'отказываться', 'чтолибо', 'думать', 'соответственно', 'чтото', 'делать', 'получаться', 'плохо', 'голова', 'вериться', 'фраза', 'работать', 'делать', 'хотеться', 'озвучить', 'многих', 'топика', 'хабре', 'лекарство'

In [18]:
# Функция для преобразования массива в строку
df_wth_tokens['sentence'] = df_wth_tokens['clear_text'].apply(lambda x: ' '.join(x))

In [19]:
df_wth_tokens.head(1)

,author,publication_date,hubs,comments,views,url,reading_time,individ/company,bookmarks_cnt,text_length,rating_new,text_tokens,text_pos_tags,rating_level,hubs_encoded,clear_text,sentence
0,complex,2009-08-03 14:34:35+00:00,GTD,67,6800,https://habr.com/ru/articles/66091/,2.0,individual,25.0,2027,4.0,['лишать' 'девственность' 'бложик' 'происходит...,"[NOUN, VERB, NOUN, DET, NOUN, ADV, SCONJ, PRON...",neutral,[80],"[лишать, девственность, бложик, происходить, с...",лишать девственность бложик происходить сегодн...


## N-грамы

In [40]:
def ngram(row, n):
    return list(ngrams(row, n))

# def extract_ngrams(text, n):
#     # Токенизация текста
#     tokens = nltk.word_tokenize(text)
#     # Генерация N-грамм
#     n_grams = ngrams(tokens, n)
#     # Преобразование в список
#     return list(n_grams)

In [41]:
df_wth_tokens['ngr_2'] = df_wth_tokens['clear_text'].apply(lambda x: ngram(x, 2))

In [42]:
df_wth_tokens['ngr_3'] = df_wth_tokens['clear_text'].apply(lambda x: ngram(x, 3))

In [43]:
df_wth_tokens.head(2)

,author,publication_date,hubs,comments,views,url,reading_time,individ/company,bookmarks_cnt,text_length,rating_new,text_tokens,text_pos_tags,rating_level,hubs_encoded,clear_text,ngr_2,ngr_3
0,complex,2009-08-03 14:34:35+00:00,GTD,67,6800,https://habr.com/ru/articles/66091/,2.0,individual,25.0,2027,4.0,['лишать' 'девственность' 'бложик' 'происходит...,"[NOUN, VERB, NOUN, DET, NOUN, ADV, SCONJ, PRON...",neutral,[80],"[лишать, девственность, бложик, происходить, с...","[(лишать, девственность), (девственность, блож...","[(лишать, девственность, бложик), (девственнос..."
1,popotam2,2009-07-15 20:24:31+00:00,GTD,13,3100,https://habr.com/ru/articles/64586/,1.0,individual,6.0,424,1.0,['предлагать' 'вариант' 'сделать' 'организован...,"[VERB, ADV, NUM, NOUN, VERB, PRON, ADV, ADJ, C...",neutral,[80],"[предлагать, вариант, сделать, организованный,...","[(предлагать, вариант), (вариант, сделать), (с...","[(предлагать, вариант, сделать), (вариант, сде..."


## N-gram + Naive Bayes

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [21]:
sample_df = df_wth_tokens.sample(frac=0.1, random_state=42)

In [22]:
from sklearn.metrics import accuracy_score, classification_report

### Предсказание Рейтинга

In [72]:
X = sample_df['sentence']

In [71]:
# Разделение на обучающую и тестовую выборки для предсказания рейтинга статьи
y_rating = sample_df['rating_level']
y_rating = y_rating.map({'very negative': -2, 'negative': -1, 'neutral': 0, 'positive': 1, 'very positive': 2})

In [73]:
# Разделение на обучающую и тестовую выборки для предсказания рейтинга статей
X_train, X_test, y_train_h, y_test_h = train_test_split(X, y_rating, test_size=0.25)

#### Униграмма

In [78]:
# Векторизация n-грамм
vectorizer = CountVectorizer(ngram_range=(1, 1))
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Шаг 3: Создание и обучение модели
model = MultinomialNB()
model.fit(X_train_vect, y_train_h)

# Шаг 4: Оценка модели
y_pred = model.predict(X_test_vect)

In [79]:
# Печать результата
print("Accuracy:", accuracy_score(y_test_h, y_pred))
print("Classification Report:n", classification_report(y_test_h, y_pred))

Accuracy: 0.4616473252193603
Classification Report:n               precision    recall  f1-score   support

          -2       0.00      0.00      0.00        91
          -1       0.00      0.00      0.00       220
           0       0.45      0.37      0.41      2086
           1       0.49      0.66      0.56      3083
           2       0.39      0.28      0.33      1586

    accuracy                           0.46      7066
   macro avg       0.27      0.26      0.26      7066
weighted avg       0.43      0.46      0.44      7066



/Users/vitaly/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Двуграмма

In [74]:
# Векторизация n-грамм
vectorizer = CountVectorizer(ngram_range=(2, 2))
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Шаг 3: Создание и обучение модели
model = MultinomialNB()
model.fit(X_train_vect, y_train_h)

# Шаг 4: Оценка модели
y_pred = model.predict(X_test_vect)

In [77]:
# Печать результата
print("Accuracy:", accuracy_score(y_test_h, y_pred))
print("Classification Report:n", classification_report(y_test_h, y_pred))

Accuracy: 0.4626379847155392
Classification Report:n               precision    recall  f1-score   support

          -2       0.00      0.00      0.00        91
          -1       1.00      0.01      0.03       220
           0       0.46      0.26      0.33      2086
           1       0.46      0.84      0.60      3083
           2       0.47      0.10      0.16      1586

    accuracy                           0.46      7066
   macro avg       0.48      0.24      0.22      7066
weighted avg       0.48      0.46      0.39      7066



/Users/vitaly/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Качество плохое, accuracy = 0.46

#### Триграмма

In [80]:
# Векторизация n-грамм
vectorizer = CountVectorizer(ngram_range=(3, 3))
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Шаг 3: Создание и обучение модели
model = MultinomialNB()
model.fit(X_train_vect, y_train_h)

# Шаг 4: Оценка модели
y_pred = model.predict(X_test_vect)

In [81]:
# Печать результата
print("Accuracy:", accuracy_score(y_test_h, y_pred))
print("Classification Report:n", classification_report(y_test_h, y_pred))

Accuracy: 0.4358901783187093
Classification Report:n               precision    recall  f1-score   support

          -2       0.00      0.00      0.00        91
          -1       0.55      0.03      0.05       220
           0       0.40      0.33      0.36      2086
           1       0.46      0.68      0.55      3083
           2       0.38      0.18      0.24      1586

    accuracy                           0.44      7066
   macro avg       0.36      0.24      0.24      7066
weighted avg       0.42      0.44      0.40      7066



#### униграммы, биграммы и триграммы

In [82]:
# Векторизация n-грамм
vectorizer = CountVectorizer(ngram_range=(1, 3))
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Шаг 3: Создание и обучение модели
model = MultinomialNB()
model.fit(X_train_vect, y_train_h)

# Шаг 4: Оценка модели
y_pred = model.predict(X_test_vect)

In [83]:
# Печать результата
print("Accuracy:", accuracy_score(y_test_h, y_pred))
print("Classification Report:n", classification_report(y_test_h, y_pred))

Accuracy: 0.45131616190206625
Classification Report:n               precision    recall  f1-score   support

          -2       0.00      0.00      0.00        91
          -1       1.00      0.01      0.02       220
           0       0.54      0.07      0.12      2086
           1       0.44      0.97      0.61      3083
           2       0.69      0.04      0.07      1586

    accuracy                           0.45      7066
   macro avg       0.53      0.22      0.16      7066
weighted avg       0.54      0.45      0.32      7066



/Users/vitaly/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### 1-4 граммы

In [84]:
# Векторизация n-грамм
vectorizer = CountVectorizer(ngram_range=(1, 4))
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Шаг 3: Создание и обучение модели
model = MultinomialNB()
model.fit(X_train_vect, y_train_h)

# Шаг 4: Оценка модели
y_pred = model.predict(X_test_vect)

In [85]:
# Печать результата
print("Accuracy:", accuracy_score(y_test_h, y_pred))
print("Classification Report:n", classification_report(y_test_h, y_pred))

Accuracy: 0.45075007076139256
Classification Report:n               precision    recall  f1-score   support

          -2       0.00      0.00      0.00        91
          -1       1.00      0.01      0.03       220
           0       0.55      0.06      0.11      2086
           1       0.44      0.97      0.61      3083
           2       0.72      0.04      0.07      1586

    accuracy                           0.45      7066
   macro avg       0.54      0.22      0.16      7066
weighted avg       0.55      0.45      0.32      7066



/Users/vitaly/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Выводы** 

При Naive Bayes классификации рейтинга по WEIGHTED AVG лучше всего модель по 1-4 граммам, но прирост по сравнению с 1-3 граммами всего 0.01, поэтому лучше всего использовать 1-3 граммы


### Хабы

In [23]:
df_wth_tokens.head(1)

,author,publication_date,hubs,comments,views,url,reading_time,individ/company,bookmarks_cnt,text_length,rating_new,text_tokens,text_pos_tags,rating_level,hubs_encoded,clear_text,sentence
0,complex,2009-08-03 14:34:35+00:00,GTD,67,6800,https://habr.com/ru/articles/66091/,2.0,individual,25.0,2027,4.0,['лишать' 'девственность' 'бложик' 'происходит...,"[NOUN, VERB, NOUN, DET, NOUN, ADV, SCONJ, PRON...",neutral,[80],"[лишать, девственность, бложик, происходить, с...",лишать девственность бложик происходить сегодн...


In [25]:
y_hubs = y_multi_reduced

In [26]:
y_multi_reduced

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [27]:
y = np.argmax(y_multi_reduced, axis=1)  # Получим метки: [0, 1, 2, 0, 1]

In [28]:
y

array([55, 54, 54, ..., 47, 50, 38])

In [29]:
X = sample_df['sentence']
X

215350    сегодня технопарк digital october стартовать к...
275546    оказываться doubleclick windows работет универ...
265955    год исследование специалист компания lenovo пр...
160133    шелушение кожа перхоть голова впервые столкнут...
92672     больший препятствие стоить путь рост небольшой...
                                ...                        
110085    посмотреть конфиг файрвола увидеть простыня ку...
253662    некий пользователь diggcom ник geekcapital поо...
16290     привет сегодня поговорить нужный статистика ис...
168042    представить летний лагерь ребёнок больший част...
51543     устраивалась skyeng солнце светило яркий трава...
Name: sentence, Length: 28264, dtype: object

In [30]:
# Разделение на обучающую и тестовую выборки для предсказания хабов (тем) статей
X_train, X_test, y_train_h, y_test_h = train_test_split(X, y, test_size=0.25)

#### Униграммы

In [31]:
# Векторизация текста по N-граммам
vectorizer = CountVectorizer(ngram_range=(1, 1))
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Шаг 3: Создание и обучение модели
model = MultinomialNB()
model.fit(X_train_vect, y_train_h)

# Шаг 4: Оценка модели
y_pred = model.predict(X_test_vect)

In [32]:
# Печать результата
print("Accuracy:", accuracy_score(y_test_h, y_pred))
print("Classification Report:n", classification_report(y_test_h, y_pred))

Accuracy: 0.2581375601471837
Classification Report:n               precision    recall  f1-score   support

           0       0.22      0.96      0.36      1498
           1       0.50      0.01      0.02        85
           2       0.00      0.00      0.00        25
           3       1.00      0.03      0.06        91
           4       0.76      0.10      0.17       162
           5       0.58      0.06      0.11       113
           6       1.00      0.01      0.01       151
           7       0.27      0.01      0.02       234
           8       0.80      0.09      0.16       137
           9       0.73      0.18      0.29       242
          10       0.00      0.00      0.00        97
          11       1.00      0.04      0.07        79
          12       0.72      0.18      0.29       146
          13       1.00      0.01      0.02        99
          14       0.00      0.00      0.00        71
          15       0.00      0.00      0.00        45
          16       0.00     

/Users/vitaly/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Двуграммы

In [96]:
# Векторизация текста по N-граммам
vectorizer = CountVectorizer(ngram_range=(2, 2))
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Шаг 3: Создание и обучение модели
model = MultinomialNB()
model.fit(X_train_vect, y_train_h)

# Шаг 4: Оценка модели
y_pred = model.predict(X_test_vect)

In [97]:
# Печать результата
print("Accuracy:", accuracy_score(y_test_h, y_pred))
print("Classification Report:n", classification_report(y_test_h, y_pred))

Accuracy: 0.27512029436739316
Classification Report:n               precision    recall  f1-score   support

           0       0.23      0.98      0.38      1515
           1       0.71      0.06      0.11        82
           2       0.00      0.00      0.00        20
           3       1.00      0.04      0.08        93
           4       0.75      0.10      0.18       150
           5       0.62      0.04      0.08       116
           6       0.38      0.04      0.08       137
           7       0.41      0.03      0.06       236
           8       0.56      0.07      0.12       129
           9       0.59      0.11      0.18       219
          10       0.71      0.04      0.08       113
          11       0.80      0.04      0.08        95
          12       0.73      0.16      0.26       173
          13       0.56      0.05      0.09       106
          14       0.50      0.01      0.03        68
          15       0.50      0.04      0.07        50
          16       0.50    

/Users/vitaly/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Триграммы

In [35]:
# Векторизация текста по N-граммам
vectorizer = CountVectorizer(ngram_range=(3, 3))
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Шаг 3: Создание и обучение модели
model = MultinomialNB()
model.fit(X_train_vect, y_train_h)

# Шаг 4: Оценка модели
y_pred = model.predict(X_test_vect)

In [36]:
# Печать результата
print("Accuracy:", accuracy_score(y_test_h, y_pred))
print("Classification Report:n", classification_report(y_test_h, y_pred))

Accuracy: 0.30667987545994907
Classification Report:n               precision    recall  f1-score   support

           0       0.25      0.84      0.38      1498
           1       0.33      0.15      0.21        85
           2       0.00      0.00      0.00        25
           3       0.57      0.14      0.23        91
           4       0.54      0.14      0.22       162
           5       0.54      0.17      0.26       113
           6       0.35      0.11      0.17       151
           7       0.29      0.10      0.15       234
           8       0.62      0.23      0.34       137
           9       0.52      0.26      0.34       242
          10       0.61      0.11      0.19        97
          11       0.68      0.24      0.36        79
          12       0.47      0.43      0.45       146
          13       0.50      0.08      0.14        99
          14       0.38      0.04      0.08        71
          15       0.71      0.22      0.34        45
          16       0.57    

/Users/vitaly/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### 1-2 граммы

In [33]:
# Векторизация текста по N-граммам
vectorizer = CountVectorizer(ngram_range=(1, 2))
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Шаг 3: Создание и обучение модели
model = MultinomialNB()
model.fit(X_train_vect, y_train_h)

# Шаг 4: Оценка модели
y_pred = model.predict(X_test_vect)

In [34]:
# Печать результата
print("Accuracy:", accuracy_score(y_test_h, y_pred))
print("Classification Report:n", classification_report(y_test_h, y_pred))

Accuracy: 0.2230399094254175
Classification Report:n               precision    recall  f1-score   support

           0       0.21      1.00      0.35      1498
           1       1.00      0.01      0.02        85
           2       0.00      0.00      0.00        25
           3       0.00      0.00      0.00        91
           4       1.00      0.01      0.02       162
           5       1.00      0.02      0.03       113
           6       0.80      0.03      0.05       151
           7       1.00      0.00      0.01       234
           8       1.00      0.02      0.04       137
           9       0.78      0.03      0.06       242
          10       0.00      0.00      0.00        97
          11       0.00      0.00      0.00        79
          12       0.75      0.04      0.08       146
          13       1.00      0.01      0.02        99
          14       0.00      0.00      0.00        71
          15       0.00      0.00      0.00        45
          16       0.00     

/Users/vitaly/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### 1-3 граммы

In [37]:
# Векторизация текста по N-граммам
vectorizer = CountVectorizer(ngram_range=(1, 3))
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Шаг 3: Создание и обучение модели
model = MultinomialNB()
model.fit(X_train_vect, y_train_h)

# Шаг 4: Оценка модели
y_pred = model.predict(X_test_vect)

In [38]:
# Печать результата
print("Accuracy:", accuracy_score(y_test_h, y_pred))
print("Classification Report:n", classification_report(y_test_h, y_pred))

Accuracy: 0.2265779790546278
Classification Report:n               precision    recall  f1-score   support

           0       0.22      1.00      0.35      1498
           1       1.00      0.01      0.02        85
           2       0.00      0.00      0.00        25
           3       1.00      0.01      0.02        91
           4       1.00      0.02      0.04       162
           5       1.00      0.03      0.05       113
           6       0.86      0.04      0.08       151
           7       1.00      0.01      0.03       234
           8       1.00      0.02      0.04       137
           9       0.85      0.05      0.09       242
          10       0.00      0.00      0.00        97
          11       1.00      0.01      0.02        79
          12       0.80      0.03      0.05       146
          13       1.00      0.01      0.02        99
          14       0.00      0.00      0.00        71
          15       1.00      0.02      0.04        45
          16       0.00     

/Users/vitaly/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Вывод**

Лучше всего по качеству предсказание по 1-3 граммам, но оптимально по качеству и скорости использовать двуграмму